## <font color='maroon'> WARNING:

#### <font color='blue'>Script to download and quantize LLM

In [1]:
# ---------------------------------------------------------------------------------------------
# IMPORTANT:  THE FOLLOWING SCRIPT USES RH3 (RR_ANALYST): SAP HANA DATABASE FOR QUERYING DATA
# ---------------------------------------------------------------------------------------------

# # Import packages
import re
import pandas as pd
import numpy as np
import time
script_start_time = time.time()

# Importing the necessary libraries
import numpy as np
import pandas as pd
# set max_rows option to None
pd.options.display.max_rows = None      # To display all the requested rows
# Set the maximum number of columns to display
pd.set_option('display.max_columns', None)
# Set the maximum width of each column to display
pd.set_option('display.max_colwidth', None)

import os
import concurrent.futures   # For parallel computing
import logging
logging.basicConfig(level=logging.INFO)
import urllib.request
import io
import os
import shutil
import concurrent.futures               # For Multi-Threading
import logging                          # For Logging info logs
logging.basicConfig(level=logging.INFO) # Set logging level
import numpy as np
import datetime
import pytz                             # For current datetime in IST zone
import os
import time
import numpy as np
import time
import glob
import random
random.seed(0)
np.random.seed(0)

import datetime
from datetime import date, timedelta
import numpy as np
import pandas as pd
import time
from datetime import date

import os
# os.getcwd()
os.environ['CURL_CA_BUNDLE'] = ''

import torch
import time
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

#### <font color='blue'>1. Downloading and Saving [ONLINE VERSION]

In [2]:

# =================================================================================
# EXECUTE ONLY DURING FIRST RUN
# =================================================================================


# Hugging Face Authentication (Token: Read permissions)
# Replace the following string with your actual Hugging Face authentication token
hf_token = <ADD_YOUR_HUGGING_FACE_TOKEN_HERE>   # EDIT IF REQUIRED

# Filepath to save LLM
# model_tokenizer_filepath = '/content/Mistral-7B-v0.2-Model-Files/'  # EDIT IF REQUIRED
model_tokenizer_filepath = './Mistral-7B-v0.2-Model-Files/'  # EDIT IF REQUIRED
# =================================================================================

# =================================================================================
# ---------------------------------------------------------------------------------------------
# Suppress all warnings
import warnings
from langchain_core._api.deprecation import LangChainDeprecationWarning
warnings.filterwarnings("ignore")
warnings.filterwarnings('ignore', category=FutureWarning)
# Suppress specific LangChainDeprecationWarning
warnings.filterwarnings("ignore", category=LangChainDeprecationWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning, module='langchain_core._api.deprecation')

# HuggingFace Authentication (DEFINED PRIOR. STORED IN 'hf_token')
login(token=hf_token)

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("*" * 70)
print(f"Device: {device}")
print("*" * 70)

import gc
gc.collect()

import torch
if device=='cuda':
    torch.cuda.empty_cache()


**********************************************************************
Device: cuda
**********************************************************************


In [3]:
!pip install -U bitsandbytes

In [4]:
# ------------------------ STEPS TO DOWNLOAD AND SAVE THE MODEL ------------------------------
# 1. Download MODEL and TOKENIZER files ahead of time with PreTrainedModel.from_pretrained():
# --------------------------------------------------------------------------------------------
# Loading the 4bit model and Tokenizer
a = time.time()

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# 4-bit quantization:
# Define Quantization Config
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Downloading 'model_4bit' and 'tokenizer'
if 'model_4bit' not in globals():
    model_4bit = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map={"": device},
        quantization_config=quantization_config
    )

if 'tokenizer' not in globals():
    tokenizer = AutoTokenizer.from_pretrained(model_name)


# 2. Save the downloaded files to a specified directory with PreTrainedModel.save_pretrained():
tokenizer.save_pretrained(model_tokenizer_filepath)  # './Mistral-7B-v0.2-Model-Files/'
model_4bit.save_pretrained(model_tokenizer_filepath)

b = time.time()
print("*" * 70)
print(f"Time required for loading and saving the 4bit model and tokenizer: {round((b - a) / 60, 2)} minutes.")
print("*" * 70)

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

**********************************************************************
Time required for loading and saving the 4bit model and tokenizer: 8.54 minutes.
**********************************************************************


In [6]:
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00


#### <font color='blue'>2. Inferencing (USING ONLINE VERSION)

In [11]:
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import PromptTemplate, LLMChain

text_corpus = "INDIA"
prompt_templates = {'prompt_template_1': '''[INST] Write an essay of 250 words on {text_corpus}. Make sure to complete the sentence at the end of the essay [/INST]'''}

# ========================================================================================================================

# ====================================================================================================
# ====================================================================================================
# ====================================================================================================

# Defining a function to instantiate a LLM pipeline basis selected prompts

def pipeline_instantiate(model_4bit, tokenizer, MAX_NEW_TOKENS):
    """
    This Python function, pipeline_instantiate, is designed to create and return a text generation pipeline instance.
    Let's break down the arguments and parameters:

    "text-generation":
        This is the type of pipeline. In this case, it's a text generation pipeline.

    model=model_4bit:
        This is the model that the pipeline will use. Here, the model_4bit is a pre-trained LLM (Mistral-7B-Instruct-v0.2 LLM).

    tokenizer=tokenizer:
        This is the tokenizer that the pipeline will use. It's the tokenizer belongs to the respective model_4bit  (pre-trained LLM)

    use_cache=True:
        This means that the pipeline will use a cache of store previously computed results. This will speed up the computation.

    trust_remote_code=True:
        This means that the pipeline will trust and execute remote code, which can be risky if the remote code is not secure.

    device_map={"": device}:
        This is a dictionary that maps pipeline components to devices. Here, all components are mapped to the device variable.

    max_new_tokens=MAX_NEW_TOKENS:
        This is the maximum number of new tokens that the model will generate.

    num_return_sequences=1:
        This means that the pipeline will return only one generated sequence.

    eos_token_id=tokenizer.eos_token_id:
        This is the ID of the end-of-sequence token.

    pad_token_id=tokenizer.eos_token_id:
        This is the ID of the padding token. Here, it's set to the same value as the end-of-sequence token, which is not a common practice.

    The function returns the instantiated pipeline, pipeline_inst.
    """
    pipeline_inst = pipeline(
            "text-generation",
            model=model_4bit,
            tokenizer=tokenizer,
            use_cache=True,
            trust_remote_code=True,
            device_map={"": device},
            max_new_tokens=MAX_NEW_TOKENS,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
    return pipeline_inst

llm = HuggingFacePipeline(pipeline=pipeline_instantiate(model_4bit=model_4bit,
                                                        tokenizer=tokenizer,
                                                        MAX_NEW_TOKENS=1000))


# ========================================================================

import gc
import torch

def generate_response(prompt_template, text_corpus, llm):
    """
    Generates a response based on the given prompt template and text corpus using a large language model (LLM).

    This function constructs a prompt using the provided template and text corpus, runs the prompt
    through the specified language model (LLM), and returns the generated response. After generating
    the response, the function clears unused variables and frees up CUDA memory.

    Args:
        prompt_template (str): The template string for the prompt, with placeholders for input variables.
        text_corpus (str): The input text corpus to be used in the prompt.
        llm (LLM): The language model to generate the response.

    Returns:
        str: The generated response from the language model.
	"""
    prompt = PromptTemplate(template=prompt_template,
                          template_format="f-string",
                          input_variables=["text_corpus"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run({"text_corpus": text_corpus})

    # Delete variables that are not required
    del prompt
    del llm_chain
    import gc
    gc.collect()

    # Deleting dedicated memory occupied by CUDA (post prompt execution)
    torch.cuda.empty_cache()

    return response

# ========================================================================
# # Defining a function to get rid of instruction part from the output sequence
# (The output sequence ahs instruction + generated text)
def remove_instructions(text):
    """
    Remove the instruction part from the input text (which is the output sequence from LLM)
    and return the cleaned text (generated response only).

    Parameters:
    text (str): The input text containing instructions and generated content.

    Returns:
    str: The text (output sequence/response) with instructions removed, trimmed of leading and trailing whitespace and newlines.
    """
    # Remove instructions + leading and trailing whitespace and newlines
    cleaned_text = text.split("[/INST]")[-1].strip()
    return cleaned_text

# ================================================================================
response_with_instructions = generate_response(prompt_templates['prompt_template_1'], text_corpus, llm)
output_online = remove_instructions(response_with_instructions)

print("\n\n")
print(output_online)

Device set to use cuda





Title: India: A Vibrant Mosaic of Unity in Diversity

India, the land of diverse cultures, traditions, and religions, is a mosaic of unity in diversity. This vast and populous country, located in South Asia, is known for its rich history, colorful festivals, and warm hospitality. India is a land of contrasts, where modernity coexists with tradition, and progress intertwines with history.

India's history is a long and complex one, with roots dating back to ancient civilizations such as the Indus Valley Civilization and the Mauryan Empire. Over the centuries, India has been influenced by various invaders and rulers, including the Mughals, the British, and the Portuguese. These influences have left an indelible mark on India's culture, language, and cuisine.

India is a melting pot of various religions, including Hinduism, Islam, Christianity, Sikhism, Buddhism, and Jainism. Each religion has its unique customs, traditions, and festivals, adding to the richness and diversity of Indian